In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession 

sc = SparkContext()
spark = SparkSession.builder.appName("HW4-1").getOrCreate()

In [2]:
spotify = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/training_set/log_0_20180715_000000000000.csv")

In [4]:
feature = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/track_features")

## Merge Data

In [5]:
df = spotify.join(feature, spotify.track_id_clean == feature.track_id , "inner")
df.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [6]:
df.count()

2990609

In [7]:
df.na.drop().count()

2990609

In [10]:
df.cache()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

## Change Data Type

In [ ]:
columns_to_drop = ["date", "premium", "vendor_id", "pickup_datetime", "dropoff_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "total_amount", "tip_amount"]
df_after = df_after.drop(*columns_to_drop)

In [ ]:
from pyspark.sql.types import *

for c in ["skip_1", "skip_2", "skip_3", "not_skipped", "context_switch", "no_pause_before_play",
         "short_pause_before_play", "long_pause_before_play", "hist_user_behavior_is_shuffle"]:
    df = df.withColumn(c, df[c].cast(BooleanType()))
for c in ["session_position", "session_length", "hist_user_behavior_n_seekfwd", "hist_user_behavior_n_seekback", "hour_of_day"]:
    df = df.withColumn(c, df[c].cast(IntegerType()))
for c in ["duration", ""]:
    df = df.withColumn(c, df[c].cast(FloatType()))   
    
df.printSchema()